# Implémentation et test du modèle XGBOOST
---

Dans ce notebook, on va découvrir les différents aspects du modèle XGBOOST. En premier lieu nous allons implémenter le modèle à zéro (from scratch) ensuite en deuxième partie nous allons directement utiliser le modèle depuis l'officielle bibliothèque ([xgboost](https://github.com/dmlc/xgboost)) et effectuer avec différents tests.

In [6]:
# Importation des modules nécessaires

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.__version__, pd.__version__

('1.24.3', '2.0.1')

## Introduction

Pour bien débuter, nous posons d'abord les premières notions qu'on va utiliser directement dans les prochaines sections.
Tout au long du notebook, en parlant du dataset donné on note le nombre d'individus **$n$** et le nombre de caractéristiques de chaque individu **$m$**. Ou plus explicitement le dataset (**$D$**) est donné comme il suit:
$$D = \{(x_i,y_i) \text{ tq } x_i \in R^m, y_i \in R\}$$

## I. Réalisation des algorithmes

Cette partie servira à la compréhension des algorithmes utilisés dans un modèle XGBOOST en les implémentant de zéro pour les deux cas (Régression et Classification). Pour cela nous allons utiliser seulement la bibliothèque **numpy** qui est utile dans les calculs matriciels.

### I.1. Fonction de prédiction

Pour un dataset donnée (**$D$**), la fonction de prédiction du modèle est la somme des $K + 1$ plus simples fonctions de prédiction. Les fonctions de prédiction ($f_k$) avec $k >= 1$ donne les résultats du parcours des arbres de régression construits lors de l'entrainement du modèle. Pour la fonction initiale ($f_0$), ce n'est qu'une valeur constante initiale déterminée au début de l'entraînement. On donne la fonction de prédiction du modèle donc comme il suit:
$$\hat{y_i} = \phi(x_i) = f_0 + \eta \sum\limits_{k = 1}\limits^{K} f_k(x_i)$$
Note: **$\eta$** représente le taux d'apprentissage (learning rate)

In [7]:
def pred(X, f0, fs, eta=0.3):
    return f0 + eta * np.sum(np.array([fk(X) for fk in fs]), axis=0)

def f_test(X):
    return np.sum(np.square(X) + X + 4, axis=1)

X = np.array(
    [
        [1, 3],
        [4, 5]
    ]
)

f0 = 3.4

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([16.6, 38.2])
#---------------------------------------------------------------------

pred(X, f0, [f_test, f_test])

array([16.6, 38.2])

### I.2 Fonction du coût

L'étape suivante serait de définir la fonction du coût (loss function) pour évaluer le modèle et le faire converger vers le point optimal. Or on a deux cas: la régression et la classification. Et donc on définira pour chacun des cas sa fonction de coût ensuite leurs dérivées pour pouvoir appliquer le [gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) par la suite. 

#### I.2.1 Fonction du coût (Régression)

Pour la régression, nous allons utiliser la somme des erreurs carrées (sum of squares error, SSE). Elle est définie comme il suit
$$SSE = l(y_i,\hat{y_i}) = \frac{1}{2} (y_i - \hat{y_i})^2$$
Note: la fraction $\frac{1}{2}$ est ajoutée juste pour simplifier les calculs (notamment pour la dérivée)

In [8]:
def SSE(Y, Y_pred):
    return (1/2) * np.square(Y - Y_pred)

Y = np.array([1.2, 3.5, 6.7])
Y_pred = np.array([1.0, 4, 7.2])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([0.02 , 0.125, 0.125])
#---------------------------------------------------------------------

SSE(Y, Y_pred)

array([0.02 , 0.125, 0.125])

Ensuite on calcule la dérivée de cette fonction de coût par rapport aux prédictions réalisées. Cela nous permettra de trouver la prédiction qui minimise la fonction coût. La dérivée est donc donnée comme il suit:
$$\frac{\partial SSE}{\partial \hat{y_i}} = \hat{y_i} - y_i$$
Note: La valeur $y - \hat{y_i}$ est appelée le résidu **$r_{im}$** où $m$ représente le numéro de l'itération. Donc on a:
$$r_{im} = - \frac{\partial SSE}{\partial \hat{y_i}}$$

In [9]:
def dSSE(Y, Y_pred):
    return Y_pred - Y

Y = np.array([1.2, 3.5, 6.7])
Y_pred = np.array([1.0, 4, 7.2])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([-0.2,  0.5,  0.5])
#---------------------------------------------------------------------

dSSE(Y, Y_pred)

array([-0.2,  0.5,  0.5])

#### I.2.1 Fonction du coût (Classification)

Pour la classification (dans ce cas binaire), nous allons utiliser l'inverse du log de vraisemblance (negative log-likelihood). Elle est définie comme il suit
$$NLL = l(y_i,p_i) = -\text{ }(\text{ }y_i \log(p_i) + (1-y_i) \log(1-p_i)\text{ })$$
Note: **$p_i$** est la probabilité prédite par le modèle. Une valeur proche de 0 signifie que le modèle classe l'individu à la classe négative et une valeur proche de 1 signifie que le modèle classe l'individu à la classe positive.

In [13]:
def NLL(Y, Y_pred):
    return - (Y * np.log(Y_pred) + (1 - Y) * np.log(1 - Y_pred))

Y = np.array([1, 0, 1])
Y_pred = np.array([0.7, 0.2, 0.8])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([0.35667494, 0.22314355, 0.22314355])
#---------------------------------------------------------------------

NLL(Y, Y_pred)

array([0.35667494, 0.22314355, 0.22314355])

Ensuite on calcule la dérivée de cette fonction de coût par rapport au log de chance de gain (log of odds). Cela nous permettra de trouver la prédiction qui minimise la fonction coût. On rappele que la chance de gain (odds) d'un évènement de propababilité $p$ est définie par:
$$odds = \frac{p}{1 - p}$$
$$\log(odds) = \log(\frac{p}{1 - p})$$
Dériver par rapport au $\log(odds)$ est aussi un choix minutieux pour réduire et faciliter les calculs pour le modèle. Il est permis de faire ce choix car il y a une relation directe entre le $\log(odds)$ et la probabilité $p$. En effet, On peut récupérer la probalité $p$ depuis $\log(odds)$ ainsi:
$$p = \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}$$
Calculons maintenant la dérivée par rapport au $\log(odds)$:
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -(\text{ }y_i \log(p_i) + (1-y_i) \log(1-p_i)\text{ })$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(p_i) + y_i \log(1-p_i) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i (\log(p_i) - \log(1-p_i)) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(\frac{p_i}{1-p_i}) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(odds) - \log(1-p_i)$$
On a:
$$\log(1-p_i) = \log(1 - \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}) = \log(\frac{1 + e^{\log(odds)}}{1 + e^{\log(odds)}} - \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}) = \log(\frac{1}{1 + e^{\log(odds)}})$$
$$\log(1-p_i) = -\log(1 + e^{\log(odds)})$$
Et donc:
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(odds) + \log(1 + e^{\log(odds)})$$
$$\frac{\partial NLL}{\partial \log(odds)} = -y_i + \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}$$
$$\frac{\partial NLL}{\partial \log(odds)} = p_i - y_i$$

In [14]:
def dNLL(Y, Y_pred):
    return Y_pred - Y

Y = np.array([1, 0, 1])
Y_pred = np.array([0.7, 0.2, 0.8])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([-0.3,  0.2, -0.2])
#---------------------------------------------------------------------

dNLL(Y, Y_pred)

array([-0.3,  0.2, -0.2])